## Análisis de Componentes Principales

Cuando tenemos muchas columnas de una matriz de datos, el entrenamiento puede tornarse infructuoso dado que se hacen muchos cálculos, más gasto de tiempo y más demora para llegar a una convergencia.

Supongamos que tenemos una matriz de datos de dimensión $m,n$, donde $m$ es la cantidad de filas o ejemplos para entrenar los datos y $n$ es la cantidad de columnas ó características que tiene la matriz. Cuando $n$ es tan grande que aumenta el procesamiento y no da para lograr entrenar rápido y efectivamente a un modelo, el PCA o análisis de componentes principales puede ser útil.

PCA consiste en:
- Previamente estandarizar los datos, si lo amerita.
- Crear la matriz de covarianza entre los datos. Es lógico que la matriz de covarianza será de dimensión $n,n$
- Buscar los autovalores y autovectores que satisfagan a la matriz de covarianza
- Organizar los autovalores de mayor a menor
- Dada una selección previa de los componentes finales que queremos obtener, se escogen la cantidad de autovalores.
- Luego de ya obtenida la matriz de autovectores, se hace un producto de matrices de la matriz de datos, de dimensión $m,n$ ya estandarizada por los autovectores, de dimensión $n,z$, donde $z < n$, lógicamente la matriz resultante es de dimensión $m,z$ y tendrá por significado la matriz de datos en modo PCA.